In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from rcrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_rcrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(31943, 1035)

In [6]:
docs_bow = [instance.bow for instance in instances_train]
docs_raw = [[[bow_index]*int(doc_bow[bow_index]) for bow_index in np.where(doc_bow > 0)[0]] for doc_bow in docs_bow]
docs_words = [[idx for idxs in doc for idx in idxs] for doc in docs_raw]
np.sum([len(doc_words) for doc_words in docs_words])

568401

# run

## initialize log

In [7]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [8]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=1, config=config)
train_docs = get_docs(instances_train, config, topic_root=topic_root)
dev_docs = get_docs(instances_dev, config, topic_root=topic_root)
test_docs = get_docs(instances_test, config, topic_root=topic_root)
init(train_docs, dev_docs, test_docs, topic_root)

## run

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
#     ppl_train = get_perplexity(train_docs, topic_root)
    ppl_train = 0.
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2] if 2 in depth_spec else 0, '%.2f'%depth_spec[3] if 3 in depth_spec else 0, \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series
    display(log_df)
    get_freq_tokens_rcrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   157   0  0      0    465   460  0.05  0.52  0.71  0.60  0.21
1   302   1  0      0    410   399  0.05  0.51  0.69  0.52  0.22
2   320   2  0      0    380   370  0.05  0.47  0.70  0.50  0.20
3   321   3  0      0    368   361  0.05  0.46  0.73  0.46  0.20
4   360   4  0      0    361   354  0.05  0.49  0.73  0.46  0.20
5   367   5  0      0    356   352  0.05  0.50  0.72  0.47  0.20
6   389   6  0      0    353   347  0.05  0.49  0.72  0.42  0.21
7   393   7  0      0    348   342  0.05  0.49  0.71  0.44  0.21
8   398   8  0      0    346   342  0.05  0.49  0.72  0.47  0.21
9   399   9  0      0    342   341  0.05  0.49  0.73  0.47  0.20
10  401  10  0      0    340   340  0.05  0.49  0.73  0.45  0.20
11  405  11  0      0    339   341  0.05  0.49  0.73  0.42  0.20
12  389  12  0      0    336   337  0.05  0.48  0.74  0.47  0.19
13  381  13  0      0    334   335  0.05  0.49  0.74  0.47  0.19
14  390  14  0      0    333   333  0.05  0.49  0.74  0.49  0.19
15  363  15  0      0    334   333  0.05  0.49  0.73  0.47  0.19
16  380  16  0      0    332   327  0.05  0.49  0.74  0.44  0.18
17  383  17  0      0    330   329  0.05  0.50  0.76  0.42  0.17
18  375  18  0      0    330   329  0.05  0.50  0.75  0.45  0.18
19  366  19  0      0    328   324  0.05  0.50  0.76  0.44  0.17
20  374  20  0      0    326   325  0.05  0.49  0.76  0.47  0.17
21  376  21  0      0    324   322  0.05  0.49  0.75  0.48  0.17
22  352  22  0      0    326   322  0.05  0.50  0.76  0.48  0.16
23  358  23  0      0    325   322  0.05  0.50  0.77  0.49  0.16
24  343  24  0      0    325   322  0.05  0.50  0.77  0.50  0.16
25  361  25  0      0    321   320  0.05  0.50  0.75  0.50  0.16
26  337  26  0      0    323   320  0.05  0.50  0.76  0.49  0.16
27  340  27  0      0    325   320  0.05  0.50  0.75  0.50  0.16
28  335  28  0      0    322   320  0.05  0.50  0.76  0.49  0.16
29  340  29  0      0    322   320  0.05  0.50  0.76  0.49  0.16

   0 : ['0-1', '0-2', '0-3', '0-4', '0-5', '0-6', '0-7', '0-8', '0-9', '0-10', '0-11', '0-12', '0-13', '0-14', '0-15', '0-16', '0-17', '0-18', '0-19', '0-20', '0-21', '0-22'] 886 15018.0 ['!', 'love', 'cover', 'nice', 'sleeve', ';', 'carry', '&', 'bought', 'price']
     0-1 : ['0-1-1', '0-1-3'] 4784 59202.0 ['sleeve', '&', ';', 'protection', 'pro', 'inside', 'air', 'nice', '!', 'zipper']
       0-1-1 : [] 456 3900.0 ['&', ';', '!', 'big', 'sleeve', 'bought', 'air', 'laptops', 'small', 'perfectly']
       0-1-3 : [] 16 257.0 ['smell', 'sleeve', 'zipper', 'neoprene', 'chemical', 'item', 'carrying', 'inside', 'sleeves', 'air']
     0-2 : [] 6749 90144.0 ['carry', '!', 'pockets', 'room', 'work', 'back', 'lot', 'love', 'books', 'space']
     0-3 : ['0-3-2'] 2450 26578.0 ['!', 'smell', 'return', 'item', 'received', 'ordered', 'amazon', '...', 'back', 'air']
       0-3-2 : [] 17 210.0 ['&', ';', 'seller', 'item', '-', 'wrong', 'send', 'ordered', 'cd', 'drive']
     0-4 : [] 230 2587.0 ['playe